In [ ]:
import tomotopy as tp
import spacy
from spacy.tokens import DocBin, Doc
import pandas as pd 

from tqdm import tqdm
import numpy as np
import pandas as pd
import regex as re
re.DEFAULT_VERSION = re.VERSION1
import plotly.express as px
from IPython.display import Image

import time
import gc

## Apply NLP pipeline for tokenization, lemmatization and other features for latter uses

In [ ]:
doc_bin = DocBin(store_user_data=True)
Doc.set_extension("ID", default=None)
nlp = spacy.load('fr_core_news_lg', exclude=["ner"])
texts = (text, ID for ID, text for textDf.content.iteritems())
for doc, ID in nlp.pipe(texts,as_tuples = True, batch_size=512,n_process=24):
    doc._.ID = ID
    doc_bin.add(doc)
    
doc_bin.to_disk("data/le_figaro.spacy")
    

# Pachinko allocation

In [ ]:
nlp = spacy.load('fr_core_news_lg', disable=["ner"])

Doc.set_extension("ID", default=None)
doc_bin = DocBin(store_user_data=True).from_disk("./data.spacy")

In [ ]:
def preprocess_text(doc):
    ID = doc._.ID
    doc = [doc.lemma_.lower() for word in doc 
           if word.is_alpha and (not word.is_stop) and (len(word.lemma_)>2) and (not word.is_oov)] 
    return ID, text

In [ ]:
docs = ((ID,preprocess_text(doc)) for ID, doc 
        in tqdm(doc_bin.get_docs(nlp.vocab)))
textDf = pd.DataFrame(docs, columns=['index','lemmatized'])


In [128]:
a = 2
(a*6)*5

60

In [ ]:
bigram = Phrases(raw_articles, min_count=10, threshold=10)
bigram_mod = Phraser(bigram)
raw_articles = list(bigram_mod[raw_articles])
trigram = Phrases(raw_articles, min_count=10, threshold=15)
trigram_mod = Phraser(bigram)
raw_articles = list(trigram_mod[raw_articles])
textDf["nGram"] = raw_articles
textDf["nGram"] = textDf["nGram"].apply(' '.join)
textDf["lemmatize"] = textDf["lemmatize"].apply(' '.join)
textDf.to_csv('data/le_figaro_lemmatized_without_stop.csv',index=False)

In [ ]:
textDf = pd.read_csv('data/le_figaro_lemmatized_without_stop.csv')
raw_articles = textDf.nGram.str.split().to_list()

In [ ]:
corpus = tp.utils.Corpus()
for doc in raw_articles:
    if doc:
        corpus.add_doc(doc)

In [ ]:
k1_max = 8 
k2_max = 20
k2_min = 5
params = []
scores = []
num_iter = 0
max_iter = 0
for k1 in range(1, k1_max):
    for k2 in range(max(k1,k2_min),k2_max):
        max_iter +=1
        
start = time.time()
for k1 in range(1, k1_max):
    for k2 in range(max(k1,k2_min),k2_max):
        num_iter+=1
        model = tp.PAModel(tw=tp.TermWeight.IDF, min_df=10, rm_top=25, k1=k1, k2=k2, corpus=corpus)
        model.burn_in = 15
        model.train(40, workers=24)
        score = tp.coherence.Coherence(model, coherence="c_v").get_score()
        params.append((k1,k2))
        scores.append(score)
        print("Runtime: %.2f seconds" %(time.time() - start), "|| Number of Searches: %s out of  %s" %(num_iter, max_iter), "|| k1: %s & k2: %s || coherence : %.3f" %(k1,k2,score))

best_params = np.argmax(scores)
print("(BEST MODEL) k1=%s k2=%s coherence=%.2f"  %(params[best_params][0], params[best_params][1],  scores[best_params]))

# BERTopic

In [1]:
import os, json
import pandas as pd
import json 
import plotly.express as px
from IPython.display import Image

import regex as re
re.DEFAULT_VERSION = re.VERSION1

import gc
import string
import time
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.models.callbacks import CallbackAny2Vec
from polyleven import levenshtein
import spacy
import numpy as np
import pickle

text = []
for root, dirs, files in os.walk('data/le_figaro'):
    for name in files:
        if name.endswith((".json")) and 'le_figaro' not in name:
            with open(os.path.join(root, name)) as f: 
                text.append(json.load(f))


In [2]:
t = time.time()
textDf = pd.DataFrame(text)
gc.collect()
textDf['date'] = textDf.date.apply(lambda x : x[0]) 
textDf['year'] = textDf.date.apply(lambda x : x.split('-')[0]) 
textDf['nPages'] = textDf.contentAsText.apply(len)
textDf = textDf[['contentAsText','date','nPages']].rename(columns = {'contentAsText':'pages'})
textDf = textDf.explode('pages')
gc.collect()
print(time.time()-t)

1.527505874633789


In [31]:
minimalSet = lambda s: {elem for elem in s if len(elem)>0 and not any(el in elem for el in s if el!=elem)} 
minimalSet({})

set()

In [32]:
any(())

False

In [38]:
paragraphDf = textDf.copy()
paragraphDf['pages'] = paragraphDf.pages.str.split('\n')
paragraphDf = paragraphDf.explode('pages')

In [39]:
paragraphDf['len'] = paragraphDf['pages'].apply(len)

In [40]:
paragraphDf = paragraphDf[paragraphDf.len>=5].copy()
keepLetter = lambda p : re.sub('[^\p{L}]','',p).lower()
paragraphDf['charBlock'] = paragraphDf.pages.apply(keepLetter)
paragraphDf = paragraphDf[paragraphDf.charBlock.apply(len)>=5].copy()

In [42]:
paragraphDf

,pages,date,nPages,len,charBlock
0,46' Année = 3 e Série = N° 284,1900-10-21,6,30,annéeesérien
0,Le Numéro — SEME & SEINE-ET-OISE : is centimes...,1900-10-21,6,76,lenumérosemeseineetoiseiscentimesdepartementsc...
0,Dimanche 21 Octobre 1900,1900-10-21,6,24,dimancheoctobre
0,Directeurs-rGêranfs P.DE RODAYS J A. PÉRIVIEB ...,1900-10-21,6,82,directeursrgêranfspderodaysjapériviebrédacteur...
0,H. DE VILLEMESSANT Fondateur,1900-10-21,6,28,hdevillemessantfondateur
...,...,...,...,...,...
11800,"PHARMACIE NORMALE, 19,r.Drouot,Paris.",1907-05-27,6,37,pharmacienormalerdrouotparis
11800,TYROL GLACIERS.,1907-05-27,6,15,tyrolglaciers
11800,Lé plus charmant pays des Alpes. Merveilles de...,1907-05-27,6,777,lépluscharmantpaysdesalpesmerveillesdesplusmag...
11800,âSTHRSEÊSStCSEESPIS Imprimeur-Gérant QUIN TA.R...,1907-05-27,6,95,âsthrseêsstcseespisimprimeurgérantquintardpari...


In [43]:
paragraphDf['headlines'] = paragraphDf.charBlock.apply(lambda cb : [cb[:i] for i in range(5,min(25,len(cb)))])

In [44]:
headlines = paragraphDf['headlines'].explode().value_counts()

In [36]:
vowel = 'aeiouyÿŷáàâéèêìíîïòóôöüûùú'
vowel = vowel+vowel.upper()
vowel

'aeiouyÿŷáàâéèêìíîïòóôöüûùúAEIOUYŸŶÁÀÂÉÈÊÌÍÎÏÒÓÔÖÜÛÙÚ'

In [45]:
print(len(headlines))
headlines = pd.DataFrame(headlines) 
headlines['count'] = headlines.headlines
headlines = headlines[headlines['count']>50].copy()
headlines['headlines'] = headlines.index
len(headlines)

33564241


71516

In [46]:
def getRatio(headline,charBlock):
    hlLen = len(headline)
    occurences = charBlock[charBlock.apply(lambda cb : headline in cb)]
    headCount = occurences.apply(lambda cb : headline in cb[:hlLen]).sum()
    headCountbis = occurences.apply(lambda cb : headline in cb[:30]).sum()
    return (headCount/len(occurences), headCountbis/len(occurences))

headlines['ratio'] = headlines.headlines.apply(lambda x : getRatio(x,paragraphDf.charBlock))

KeyboardInterrupt: 

In [ ]:
headlines[headlines.ratio>0.8]